In [87]:
import networkx as nx
import numpy as np
import json
from collections import defaultdict

In [88]:
def load_data(ds_name, use_node_labels):
    node2graph = {}
    Gs = []

    with open("../datasets/%s/%s_graph_indicator.txt" % (ds_name, ds_name), "r") as f:
        c = 1
        for line in f:
            node2graph[c] = int(line[:-1])
            if not node2graph[c] == len(Gs):
                Gs.append(nx.Graph())
            Gs[-1].add_node(c)
            c += 1

    with open("../datasets/%s/%s_A.txt" % (ds_name, ds_name), "r") as f:
        for line in f:
            edge = line[:-1].split(",")
            edge[1] = edge[1].replace(" ", "")
            Gs[node2graph[int(edge[0])] - 1].add_edge(int(edge[0]), int(edge[1]))

    if use_node_labels:
        with open("../datasets/%s/%s_node_labels.txt" % (ds_name, ds_name), "r") as f:
            c = 1
            for line in f:
                node_label = int(line[:-1])
                Gs[node2graph[c] - 1].node[c]['label'] = node_label
                c += 1

    labels = []
    with open("../datasets/%s/%s_graph_labels.txt" % (ds_name, ds_name), "r") as f:
        for line in f:
            labels.append(int(line[:-1]))

    labels = np.array(labels, dtype=np.float)
    return Gs, labels

In [131]:
dataset = 'BZR'

In [132]:
Gs, targets = load_data(dataset, True)

In [135]:
targets = targets.astype(int)

In [136]:
node_labels = []
edge_lists = []
inverse_labels = []
for i, G in enumerate(Gs):
    labels = nx.get_node_attributes(G, 'label')
    node_labels.append({str(node): label for node, label in labels.items()})
    edge_lists.append([list(edge) for edge in G.edges()])
    inv_dict = defaultdict(list)
    for node, label in labels.items():
        inv_dict[str(label)].append(node)
    inverse_labels.append(dict(inv_dict))

In [137]:
import os
import glob

files = glob.glob('../input/'+dataset+'/train/*')
for f in files:
    os.remove(f)

In [138]:
i_train = 1
i_test = 1
for (edges, labels, inverse_labels, target) in zip(edge_lists, node_labels, inverse_labels, targets):
    to_write = {'edges': edges, 'labels': labels, 'inverse_labels': inverse_labels, 'target': int(target)}
    if np.random.rand() < 0.9:
        with open('../input/'+dataset+'/train/'+str(i_train)+'.json', 'w') as f:
            json.dump(to_write, f)
        i_train += 1
    else:
        with open('../input/'+dataset+'/test/'+str(i_test)+'.json', 'w') as f:
            json.dump(to_write, f)
        i_test += 1